In [18]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function
import os, sys
import pandas as pd
import random
import json

import numpy as np

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

batch_size = 128
num_classes = 10
epochs = 20

In [20]:
target = os.environ.get('target', 'default')

if 'ipykernel' in sys.modules:
    argv = ['', '1', 'softmax', '0.1', 'False']
else:
    argv = sys.argv

try:
    epochs = int(argv[1])
except:
    epochs = 2

hidden = 512

try:
    activate = argv[2]
except:
    activate = 'softmax'
    
try:
    dropout = float(sys.argv[3])
except:
    dropout = 0.2

print(epochs, activate, dropout)

with np.load("/mm/project/data_in/mnist.npz") as f:
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']

1 softmax 0.2


In [21]:
x_train = x_train.reshape(60000, 784)
x_train = x_train[:20000]
y_train = y_train[:20000]
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('here04')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(hidden, activation='relu', input_shape=(784,)))
model.add(Dropout(dropout))
model.add(Dense(hidden, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(num_classes, activation=activate))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),metrics=['accuracy'])

print('train on prd')
history = model.fit(x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(x_test, y_test))

print('train on prd')

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print(history.history)


20000 train samples
10000 test samples
here04
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
train on prd
Train on 20000 samples, validate on 10000 samples
Epoch 1/1
20000/20000 [====

In [24]:
arr = []
for i in range(5):
    model_path = '/mm/project/model/%s_%s.h5' % (target, "-".join(argv[1:]+[str(i)]))
    obj = {}
    obj['target'] = '%s' % (target)
    obj['args'] = " ".join(argv[1:]+[str(i)])
    obj['path'] = model_path
    obj['model'] = '%s_%s' % (target, "-".join(argv[1:]+[str(i)]))
    obj['score'] = random.random()

    model.save(model_path)

    arr.append(obj)

df = pd.DataFrame(arr)
df.to_csv('/mm/step/train.csv', index=False)

In [26]:
pd.read_csv('/mm/step/train.csv')

,args,model,path,score,target
0,1 softmax 0.1 False 0,default_1-softmax-0.1-False-0,/mm/project/model/default_1-softmax-0.1-False-...,0.550102,default
1,1 softmax 0.1 False 1,default_1-softmax-0.1-False-1,/mm/project/model/default_1-softmax-0.1-False-...,0.563255,default
2,1 softmax 0.1 False 2,default_1-softmax-0.1-False-2,/mm/project/model/default_1-softmax-0.1-False-...,0.361487,default
3,1 softmax 0.1 False 3,default_1-softmax-0.1-False-3,/mm/project/model/default_1-softmax-0.1-False-...,0.782143,default
4,1 softmax 0.1 False 4,default_1-softmax-0.1-False-4,/mm/project/model/default_1-softmax-0.1-False-...,0.495243,default
